# Resilience Against Other Distribution
This notebook investigates the resilience of the REHEATFUNQ model to the eventuality that heat flow were not gamma distributed.

In [ ]:
import cmasher
import numpy as np
from plotconfig import *
from cmcrameri.cm import *
from pickle import Unpickler
from scipy.special import erf
from cache import cached_call
import matplotlib.pyplot as plt
from numpy.typing import ArrayLike
from zeal2022hf import get_cm_colors
from scipy.interpolate import interp1d
from matplotlib.cm import ScalarMappable
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.patches import Rectangle
from reheatfunq.resilience import test_performance_mixture_cython

To this end, we determine which anomaly powers we want to investigate - the model need not be working for arbitrary powers, only for those observed on Earth. Using equation (20b) from LS80, we get for the depth-average frictional resistance for a strike-slip fault similar to the San Andreas under Byerlee conditions:
$$
\bar{R} = 445\,\mathrm{bar} = 445\times10^{5}\,\mathrm{Pa}
$$
For this frictional resistance, a $15\,\mathrm{km}$ deep fault segment of length $160\,\mathrm{km}$, and a slip rate of $5\,\mathrm{cm/yr}=1.585\times10^{-9}\,\mathrm{m/s}$, we find a maximum power dissipation of
$$
P_\mathrm{max} = 169 \,\mathrm{MW}
$$

In [ ]:
PMAX = 445e5 * 160e3 * 15e3 * 1.5854895991882295e-09
print(PMAX * 1e-6)
PMAX = 445e5 * 160e3 * 15e3 * 2.536783358701167e-09
print(PMAX * 1e-6)

Hence, we test the model's performance for the following range of powers:

In [ ]:
POWERS_MW = [10, 25, 50, 75, 102, PMAX*1e-6]

In [ ]:
Nset = np.arange(10,101,2)
#Nset = np.arange(10,101,10)

In [ ]:
M_SET = [int(M) for M in np.round(np.geomspace(5, 10000))]

The number **41** is important actually - code is compiled only for selected numbers of quantiles (4 and 41).

In [ ]:
QUANTILES = np.linspace(0.99, 0.01, 41)

##### Data from previous notebooks

In [ ]:
PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V = np.loadtxt('results/05-GCP-Parameters.txt', skiprows=1, delimiter=',')

In [ ]:
with open('intermediate/A2-Distributions-for-Resilience.pickle','rb') as f:
    inspiring = Unpickler(f).load()

##### Plot configuration:

In [ ]:
colors = get_cm_colors(vik, 7)
color0 = colors[0]
color1 = colors[4]
color2 = colors[5]

## The mixture distribution
For the three following examples of non-gamma heat flow, we use a mixture distribution of two normal distributions,
capped at $q=0$.

In [ ]:
SEED = 29181 # 29177

In [ ]:
def mixture_density(x, x0, s0, a0, x1, s1, a1):
    a0 = a0 / (a0+a1)
    a1 = 1.0 - a0
    # We simply cut the mixture distribution at x=0 and hence
    # have to renormalized by the lost mass:
    N0 = 0.5*(1.0 - erf(-x0/(s0*np.sqrt(2))))
    N1 = 0.5*(1.0 - erf(-x1/(s1*np.sqrt(2))))
    norm = a0 * N0 + a1 * N1
    return (a0*np.exp(-0.5*((x-x0)/s0)**2)/s0 + a1*np.exp(-0.5*((x-x1)/s1)**2)/s1) / (np.sqrt(2*np.pi) * norm)

In [ ]:
def test_resilience(N: ArrayLike, P_MW: ArrayLike, M: int, params: tuple[float], quantiles: ArrayLike,
                    p: float, s: float, n: float, v:float, amin=1.0, seed=SEED):
    """
    This function evaluates the performance of the REHEATFUNQ model against
    data that does not stem from a gamma distribution (modelled by a two-component
    normal mixture).
    """
    print("M =",M)
    res = [cached_call(test_performance_mixture_cython, N, M, P, *params, quantiles, p, s, n, v, amin, seed=seed)
           for P in P_MW]
    return res

## 1) Two non-overlapping normal distributions at medium heat flow

In [ ]:
I21 = 0
PARAMS_21 = (31, 3.5, 0.33, 62, 5.5, 0.67)

In [ ]:
fig = plt.figure(dpi=140, figsize=(3.684, 3.4))
ax = fig.add_subplot(111)
yp = np.linspace(20, 90, 200)
ax.plot(yp, mixture_density(yp, *PARAMS_21), label='Normal\nMixture', color=color0)
ax.hist(inspiring[0], density=True, color=color1, label='Inspiring\nSample')
ax.legend(loc='upper left');
ax.set_xlabel('Heat Flow ($\\mathrm{mW\,m}^\\mathrm{-2}$)')
ax.set_ylabel('Density ($\\mathrm{m}^\\mathrm{2}\,\\mathrm{mW}^\\mathrm{-1}$)')
fig.savefig('figures/A4-Resilience-Setup-D1.pdf')

In [ ]:
res_21 = test_resilience(Nset, POWERS_MW, M_SET[-1], PARAMS_21, QUANTILES, PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V)

In [ ]:
res_21

## 2) Two overlapping normal distributions of largely different variance

In [ ]:
I22 = 4
PARAMS_22 = (28, 2, 0.6, 22, 15, 0.4)

In [ ]:
fig = plt.figure(dpi=140, figsize=(3.684, 3.4))
ax = fig.add_subplot(111)
yp = np.linspace(0, 50, 200)
ax.plot(yp, mixture_density(yp, *PARAMS_22), label='Normal\nMixture', color=color0)
ax.hist(inspiring[1], density=True, color=color1, label='Inspiring\nSample')
ax.legend(loc='upper right');
ax.set_xlabel('Heat Flow ($\\mathrm{mW\,m}^\\mathrm{-2}$)')
ax.set_ylabel('Density ($\\mathrm{m}^\\mathrm{2}\,\\mathrm{mW}^\\mathrm{-1}$)')
fig.savefig('figures/A4-Resilience-Setup-D2.pdf')

In [ ]:
res_22 = test_resilience(Nset, POWERS_MW, M_SET[-1], PARAMS_22, QUANTILES, PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V)

## 2.3) Two non-overlapping normal distributions at larger heat flow

In [ ]:
I23 = 3
PARAMS_23 = (43, 4, 0.3, 102, 10, 0.8)

In [ ]:
fig = plt.figure(dpi=140, figsize=(3.684, 3.4))
ax = fig.add_subplot(111)
yp = np.linspace(20, 130, 200)
ax.plot(yp, mixture_density(yp, *PARAMS_23), label='Normal\nMixture', color=color0)
ax.hist(inspiring[2], density=True, color=color1, label='Inspiring\nSample')
ax.legend(loc='upper left');
ax.set_xlabel('Heat Flow ($\\mathrm{mW\,m}^\\mathrm{-2}$)')
ax.set_ylabel('Density ($\\mathrm{m}^\\mathrm{2}\,\\mathrm{mW}^\\mathrm{-1}$)')
fig.savefig('figures/A4-Resilience-Setup-D3.pdf')

In [ ]:
res_23 = test_resilience(Nset, POWERS_MW, M_SET[-1], PARAMS_23, QUANTILES, PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V)

In [ ]:
Nset

In [ ]:
ids_select = [0, 5]
all_res = (res_21, res_22, res_23)

In [ ]:
N_SELECT = 24

In [ ]:
ni = int(np.argwhere(Nset == N_SELECT))

We fill the following variables:

| Variable | Meaning |
| -------- | :------ |
|   $y$    | The tail quantile of the posterior at the true $P_H$ of the heat flow anomaly |
|   $c$    | Relative bias of the tail quantile.  |

The relative bias $c$ can be understood as follows: say we have a tail quantile $q$ (with an estimator $P_q$).
If the tail quantile were unbiased in a frequentist view, we would expect the true power $P_H$ to fall above
the tail quantile $P_q$ in a fraction $q$ of generated samples. If this is not the case, the rate of the true
$P_H$ exceeding the tail quantile $P_q$ (call this rate $r$) is not $q$ ($r\neq q$).

This allows us to
1) derive the rate $r$ of $P_H>P_q$ (this can be obtained by finding the quantile of $P_H$ in $y$)
2) determine the power $P_q'$, which is the power at the actual rejection rate $r=q$ (that is
   the power in which $P_H > P_q'$ actually occurs in a fraction $q$ of the samples)

We compute this power $P_q'$ and derive the relative bias of this power $P_q'$ compared to the
actual anomaly power $P_q$:
$$
  c = \frac{P_q'}{P_q} - 1
$$

In [ ]:
# This loop iterates the three probability densities:
y = [np.zeros((len(all_res), len(ids_select), QUANTILES.size)) for m in M_SET]
c = [np.zeros((len(all_res), len(ids_select), QUANTILES.size)) for m in M_SET]
for l,m in enumerate(M_SET):
    for k,res_k in enumerate(all_res):
        # this loop iterates the powers:
        for s,i in enumerate(ids_select):
            #i = -1
            for j,q in enumerate(QUANTILES):
                quant_ki = np.sort(res_k[i][0,ni,j,:m])
                if 1e6*POWERS_MW[i] < quant_ki[0]:
                    y[l][k,s,j] = 0.0
                elif 1e6*POWERS_MW[i] > quant_ki[-1]:
                    y[l][k,s,j] = 1.0
                else:
                    interp = interp1d(quant_ki, (np.arange(m)/(m-1)))
                    y[l][k,s,j] = interp(1e6*POWERS_MW[i])
                c[l][k,s,j] = np.quantile(quant_ki,q)/(1e6*POWERS_MW[i]) - 1.0

In [ ]:
j = [float(round(q,4)) for q in QUANTILES].index(0.108)
for l,m in enumerate(M_SET):
    for k,res_k in enumerate(all_res):
        # this loop iterates the powers:
        print("--- distribution",k,"---")
        for s,i in enumerate(ids_select):
            #i = -1
            q = 0.108
            quant_ki = np.sort(res_k[i][0,ni,j,:m])
            print("P = %3.0f" % (POWERS_MW[i],), "MW ->",
                  round(100*(np.quantile(quant_ki,q)/(1e6*POWERS_MW[i]) - 1.0)),"%")

In [ ]:
markers = ['^','s']

In [ ]:
power_scale = 1e-6

ypos = (0.13, 0.43, 0.73)[::-1]
xpos = (0.08, 0.315, 0.55, 0.81)
dx = (0.175, 0.175, 0.175, 0.175)
dy = 0.24


q_plot = 0.5

mask = Nset > 9

M = M_SET[-1]

vmin = c[-1].min()
vmax = c[-1].max()
norm = BoundaryNorm(boundaries=[-0.2, -0.1, -0.05, -0.01, 0.01, 0.1, 1.0, 5.0], ncolors=256)

with plt.rc_context({'axes.labelpad': 0.05, 'xtick.major.pad': 1.2, 'ytick.major.pad': 1.2}):
    plt.rcParams['axes.titlepad'] = 10

    fig = plt.figure(dpi=140, figsize=(6.975, 5.5))
    cax = fig.add_axes((0.75, 0.055, 0.23, 0.01), zorder=2)
    ax0 = fig.add_axes((0,0,1,1))
    ax0.set_xlim(0,1)
    ax0.set_ylim(0,1)
    highlight_color = '#eeeeee'
    ax0.add_patch(Rectangle((0.005, ypos[0] - 0.03), 0.99, 0.28, facecolor=highlight_color))
    ax0.add_patch(Rectangle((0.005, ypos[1] - 0.03), 0.99, 0.28, facecolor=highlight_color))
    ax0.add_patch(Rectangle((0.005, ypos[2] - 0.06), 0.99, 0.31, facecolor=highlight_color))
    ax0.text(0.01, ypos[0]+0.11, "D1", va='center', fontsize=10)
    ax0.text(0.01, ypos[1]+0.11, "D2", va='center', fontsize=10)
    ax0.text(0.01, ypos[2]+0.095, "D3", va='center', fontsize=10)
    ax0.set_axis_off()
    l0 = np.argmin(np.abs(QUANTILES - 0.9))
    l1 = np.argmin(np.abs(QUANTILES - 0.1))
    l2 = np.argmin(np.abs(QUANTILES - 0.5))
    l3 = np.argmin(np.abs(QUANTILES - 0.01))
    
    j = 1
    for k,res_k in enumerate(all_res):
        for s,i in enumerate(ids_select):
            ylim = (0, power_scale * res_k[i][:,mask,:,:].mean(axis=-1).max())
            ax = fig.add_axes((xpos[1 + s], ypos[k], dx[1+s], dy))
            ax.set_xscale('log')
            if k == 0:
                if i == 0:
                    ax.set_title('10 MW', pad=4)
                else:
                    ax.set_title('271 MW', pad=4)
            elif k == len(all_res)-1:
                ax.set_xlabel('Number of samples')
            if i == 0:
                ax.text(11, 0.93*ylim[1], ['(b)', '(f)', '(j)'][k], ha='center', va='center')
            else:
                ax.text(94, 0.93*ylim[1], ['(c)', '(g)', '(k)'][k], ha='center', va='center')
                
            ax.set_ylabel('Quantile $P_H$ (MW)')
            ax.set_ylim(ylim)
            # The prior:
            h0 = ax.fill_between(Nset[mask], power_scale * np.quantile(res_k[i][0,mask,l0,:], q_plot, axis=1),
                                 power_scale * np.quantile(res_k[i][0,mask,l1,:], q_plot, axis=1),
                                 color='lightblue', label='80% symmetric')
            h1 = ax.plot(Nset[mask], power_scale * np.quantile(res_k[i][0,mask,l2,:], q_plot, axis=1), label='Median')
            h2 = ax.plot(Nset[mask], power_scale * np.quantile(res_k[i][0,mask,l3,:], q_plot, axis=1),
                         color='lightblue', linestyle=':', label='Tail 1\%')
            h3 = ax.plot(Nset[mask], power_scale * POWERS_MW[i] * 1e6 * np.ones(np.count_nonzero(mask)),
                         label='True', color='k', linestyle='--', linewidth=1.0)
            for label in ax.get_yticklabels():
                label.set_rotation(90)
            j += 1

        # The QQ-plot:
        ax = fig.add_axes((xpos[3], ypos[k], dx[3], dy))
        ax.text(50, 93, ['(d)', '(h)', '(l)'][k], ha='center', va='center')
        ax.set_xlim(0, 100)
        ax.set_ylim(0, 100)
        ax.set_aspect('equal')
        ax.plot((0.0,100.0), (0.0,100.0), linewidth=1.0, color='k')
        if k == 0:
            ax.set_title(f'$N={N_SELECT}$', pad=4)
        elif k == 2:
            ax.set_xlabel('Chosen tail quantile $t$ (%)', loc='right')
        ax.set_ylabel('Rate $r$ of anomaly\nexceeding $P_H(t)$ (%)')
        ax.set_xticks((0,50,100))
        lbls = ax.set_xticklabels(('0','50','100'))
        lbls[-1].set_ha('right')
        # this loop iterates the powers:
        for s,i in enumerate(ids_select):
            hQQ = ax.scatter(100*QUANTILES, 100*y[-1][k,s,:], marker=markers[s],
                             label=str(round(POWERS_MW[i])) + " MW",
                             c = c[-1][k,s,:], s=10, #vmin=0.0, vmax=2.0,
                             cmap=cmasher.get_sub_cmap(vik, 0.15, 0.85, N=256), norm=norm,
                             edgecolor='k', linewidth=0.5)
            ax.axvline(10, linestyle='--', linewidth=1, color='k', zorder=0)

        if k == 2:
            ax.legend(fontsize=6, handletextpad=0.2, handlelength=1.0)

    # Legend axis:
    lax = fig.add_axes((0.0, 0.0, 0.7, 0.08), facecolor='none')
    lax.legend(handles = (h3[0],h0,h1[0],h2[0]),
               labels=('True', '80 % symmetric', 'Median', 'Tail 1 %'),
               ncol=4, loc='center')
    lax.grid('off')
    lax.set_axis_off()


    # The Gaussian mixture models:
    # number 1:
    gax0 = fig.add_axes((xpos[0], ypos[0]+0.04, dx[0], dy-0.04))
    xp = np.linspace(20, 90, 200)
    yp = mixture_density(xp, *PARAMS_21)
    gax0.plot(xp, yp, label='Normal\nMixture',
              color='indigo')
    gax0.set_yticks(gax0.get_yticks())
    gax0.set_yticklabels([str(int(round(100*x))) for x in gax0.get_yticks()])
    gax0.set_ylabel('Density ($10^{-2}\\mathrm{m}^2\\mathrm{mW}^{-1}$)')
    gax0.yaxis.set_label_coords(-0.12, 0.42)
    gax0.set_xlabel('Heat flow ($\\mathrm{mW}\\mathrm{m}^{-2}$)')
    gax0.text(20,4.9e-2, '$\\mu_0=31$\n$\sigma_0=3.5$\n$w_0=0.33$', fontsize=6)
    gax0.text(67,5.0e-2,'$\\mu_1=62$\n$\sigma_1=5.5$\n$w_1=0.67$', fontsize=6)
    gax0.text(50, 6.3e-2, '(a)', va='center', ha='center')
    gax0.set_ylim(0, 1.4*yp.max())
    
    
    # number 2:
    gax1 = fig.add_axes((xpos[0], ypos[1]+0.04, dx[0], dy-0.04))
    xp = np.linspace(0, 50, 200)
    yp = mixture_density(xp, *PARAMS_22)
    gax1.plot(xp, yp, label='Normal\nMixture', color='indigo')
    gax1.set_yticks(gax1.get_yticks())
    gax1.set_yticklabels([str(int(round(100*x))) for x in gax1.get_yticks()])
    gax1.set_ylabel('Density ($10^{-2}\\mathrm{m}^2\\mathrm{mW}^{-1}$)')
    gax1.yaxis.set_label_coords(-0.14, 0.42)
    gax1.set_xlabel('Heat flow ($\\mathrm{mW}\\mathrm{m}^{-2}$)')
    gax1.text(33, 10.4e-2, '$\\mu_0=28$\n$\sigma_0=2$\n$w_0=0.6$', fontsize=6)
    gax1.text(4,  1.8e-2,  '$\\mu_1=22$\n$\sigma_1=15$\n$w_1=0.4$', fontsize=6)
    gax1.text(2, 13.5e-2, '(e)', va='center', ha='center')
    gax1.set_ylim(0, 1.1*yp.max())
    
    
    # number 3:
    gax2 = fig.add_axes((xpos[0], ypos[2]+0.04, dx[0], dy-0.04))
    xp = np.linspace(20, 130, 200)
    yp = mixture_density(xp, *PARAMS_23)
    gax2.plot(xp, yp, label='Normal\nMixture', color='indigo')
    gax2.set_yticks(gax2.get_yticks())
    gax2.set_yticklabels([str(int(round(100*x))) for x in gax2.get_yticks()])
    gax2.set_ylabel('Density ($10^{-2}\\mathrm{m}^2\\mathrm{mW}^{-1}$)')
    gax2.yaxis.set_label_coords(-0.12, 0.42)
    gax2.set_xlabel('Heat flow ($\\mathrm{mW}\\mathrm{m}^{-2}$)')
    gax2.text(20,3e-2,'$\\mu_0=43$\n$\sigma_0=4$\n$w_0=0.27$', fontsize=6)
    gax2.text(90,3e-2,'$\\mu_1=102$\n$\sigma_1=10$\n$w_1=0.73$', fontsize=6)
    gax2.text(70, 3.9e-2, '(i)', va='center', ha='center')
    gax2.set_ylim(0, 1.45*yp.max())
    
    # The colorbar:
    fig.colorbar(hQQ, cax=cax, orientation='horizontal',
                 ticks=[-0.2, -0.1, -0.05, -0.01, 0.01, 0.1, 1,5])
    cax.set_xticklabels(["-20", "-10", "-5", "-1", "1", "10", "100", "500"],
                        fontsize='small');
    cax.set_xlabel('Bias $B$ at $r=t$ relative  to $P_H$ (%)');

    
    fig.savefig('figures/A4-Resilience.pdf')

### Convergence Analysis
#### 1. The QQ-Plot

In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

In [ ]:
len(all_res), len(ids_select)

In [ ]:
with plt.rc_context({'axes.labelpad': 0.05, 'xtick.major.pad': 1.2, 'ytick.major.pad': 1.2}):
    plt.rcParams['axes.titlepad'] = 10

    fig = plt.figure(dpi=140, figsize=(6.975, 3.5))
    ax_bg = fig.add_axes((0,0,1,1))
    ax_bg.text(0.015, 0.68, '10 MW', rotation=90, ha='center',
               fontsize='large')
    ax_bg.text(0.015, 0.23, '271 MW', rotation=90, ha='center',
               fontsize='large')
    ax_bg.set_axis_off()
    colors_ca = get_cm_colors(hawaii, len(M_SET)+2)

    # Color bar:
    cmap = ListedColormap(colors_ca)
    bounds = M_SET
    norm = BoundaryNorm(M_SET, cmap.N)
    cax = fig.add_axes((0.91, 0.2, 0.01, 0.7))
    fig.colorbar(ScalarMappable(cmap=cmap, norm=norm),
        cax=cax
    )
    cax.set_ylabel('Number of samples')
    for k,res_k in enumerate(all_res):
        for s,i in enumerate(ids_select):
            ax = fig.add_axes((0.09+0.28*k, 0.55 - 0.45*s, 0.28, 0.38))
            if s == 0:
                ax.set_title('D'+str(k+1))
            else:
                ax.set_xlabel('Chosen tail quantile $t$ (%)')
            # The QQ-plot:
            ax.set_xlim(0, 100)
            ax.set_ylim(0, 100)
            ax.set_aspect('equal')
            ax.plot((0.0,100.0), (0.0,100.0), linewidth=1.0, color='k')
            ax.set_ylabel('Rate $r$ of anomaly\nexceeding $P_H(t)$ (%)')
            ax.set_xticks((0,50,100))
            lbls = ax.set_xticklabels(('0','50','100'), fontsize='small')
            lbls[-1].set_ha('right')
            # this loop iterates the powers:
            for l,m in enumerate(M_SET):
                ax.plot(100*QUANTILES, 100*y[l][k,s,:], color=colors_ca[l], linewidth=1)#4 - 2/len(M_SET)*l)
            ax.axvline(10, linestyle='--', linewidth=1, color='k', zorder=0)
    
    
    fig.savefig('figures/A4-Convergence-QQ.pdf')

In [ ]:
fig = plt.figure(figsize=(12,3))
colors_ca = get_cm_colors(hawaii, len(M_SET)+2)
for k,res_k in enumerate(all_res):
    for s,i in enumerate(ids_select):
        ax = fig.add_subplot(2,6,6*s+k+1)
        # The QQ-plot:
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_ylabel(f'Standard deviation of $r$\nto value at $M={M_SET[-1]}$ (%)')
        lbls[-1].set_ha('right')
        # this loop iterates the powers:
        ax.plot(M_SET, [np.std(100*(y[l][k,s,:]-y[-1][k,s,:])) for l,_ in enumerate(M_SET)],
                marker='.')
        ax.axvline(10, linestyle='--', linewidth=1, color='k', zorder=0)


fig.tight_layout()

#### 2. The Quantiles with $N$

In [ ]:
with plt.rc_context({'axes.labelpad': 0.05, 'xtick.major.pad': 1.2, 'ytick.major.pad': 1.2}):
    plt.rcParams['axes.titlepad'] = 10

    fig = plt.figure(dpi=140, figsize=(6.975, 6.0))
    ax_bg = fig.add_axes((0,0,1,1))
    ax_bg.set_axis_off()
    
    colors_ca2 = get_cm_colors(batlow, 6)

    xpos = [0.13, 0.61]
    ypos = [0.1, 0.4, 0.7][::-1]
    dx = 0.38
    dy = 0.25
    
    j = 1
    for k,res_k in enumerate(all_res):
        ax_bg.text(0.01, ypos[k] + 0.5*dy, 'D'+str(k+1),
                   rotation=90, fontsize='large')
        for s,i in enumerate(ids_select):
            ylim = (0, power_scale * res_k[i][:,mask,:,:].mean(axis=-1).max())
            print(len(all_res), len(ids_select), k*len(ids_select)+s+1)
            #ax = fig.add_subplot(len(all_res), len(ids_select), k*len(ids_select)+s+1)
            ax = fig.add_axes((xpos[s], ypos[k], dx, dy))
            ax.set_xscale('log')
            if k == 0:
                if i == 0:
                    ax.set_title('10 MW', pad=4)
                else:
                    ax.set_title('271 MW', pad=4)
            elif k == len(all_res)-1:
                ax.set_xlabel('Number of samples $S$')
                
            ax.set_ylabel(f'Relative quantile difference\nto $M={M_SET[-1]}$')
            #ax.set_ylim(ylim)
            # The prior:
            QPH_10_base = power_scale * np.quantile(res_k[i][0,mask,l0,:], q_plot, axis=1)
            QPH_90_base = power_scale * np.quantile(res_k[i][0,mask,l1,:], q_plot, axis=1)
            QPH_50_base = power_scale * np.quantile(res_k[i][0,mask,l2,:], q_plot, axis=1)
            QPH_99_base = power_scale * np.quantile(res_k[i][0,mask,l3,:], q_plot, axis=1)
            y_10 = []
            y_90 = []
            y_50 = []
            y_99 = []
            for l,M in enumerate(M_SET[:-1]):
                QPH_10 = power_scale * np.quantile(res_k[i][0,mask,l0,:M], q_plot, axis=1)
                QPH_90 = power_scale * np.quantile(res_k[i][0,mask,l1,:M], q_plot, axis=1)
                QPH_50 = power_scale * np.quantile(res_k[i][0,mask,l2,:M], q_plot, axis=1)
                QPH_99 = power_scale * np.quantile(res_k[i][0,mask,l3,:M], q_plot, axis=1)
                y_10.append(np.std(QPH_10 - QPH_10_base) / np.mean(QPH_10_base))
                y_90.append(np.std(QPH_90 - QPH_90_base) / np.mean(QPH_90_base))
                y_50.append(np.std(QPH_50 - QPH_50_base) / np.mean(QPH_50_base))
                y_99.append(np.std(QPH_99 - QPH_99_base) / np.mean(QPH_99_base))
            ax.plot(M_SET[:-1], y_10, color=colors_ca2[0], label='10 %', marker='.', markeredgecolor='none')
            ax.plot(M_SET[:-1], y_50, color=colors_ca2[1], label='50 %', marker='.', markeredgecolor='none')
            ax.plot(M_SET[:-1], y_90, color=colors_ca2[2], label='90 %', marker='.', markeredgecolor='none')
            ax.plot(M_SET[:-1], y_99, color=colors_ca2[3], label='99 %', marker='.', markeredgecolor='none')
            mplot = np.geomspace(M_SET[0], M_SET[-1])
            ax.plot(mplot, 2/np.sqrt(mplot), color='k', linestyle='--', linewidth=1,
                    label='$2/\sqrt{S}$')
            ax.set_yscale('log')
            ax.legend(title='Tail Quantile', ncols=2)
            for label in ax.get_yticklabels():
                label.set_rotation(90)
            j += 1
    
    # fig.tight_layout()
    fig.savefig('figures/A4-Convergence-Tail-Quantiles.pdf')

### License
```
A notebook to test the gamma distribution model of regional aggregate
heat flow and its anomaly quantification capabilities against data
stemming from a different distribution.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2019-2022 Deutsches GeoForschungsZentrum Potsdam,
            2022-2023 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```